# Feature Selection

### Vif

Vif realiza una selección de características basada en el VIF para eliminar características con alta multicolinealidad, lo que ayuda a mejorar la calidad de los modelos de aprendizaje automático al reducir la redundancia entre las características.

In [ ]:

import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

class Vif(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y = None):
      X_train = X.copy()
      self.dele = []

      while True:
          VIF = np.array([variance_inflation_factor(X_train, i) for i in range(X_train.shape[1])])

          # delete the maximum one and repeat the count until the maximum value is lower than 5.
          self.dele.append(np.argmax(VIF))
          X_train = np.delete(X_train, self.dele[-1], axis=1)
          if np.max(VIF) < 5:
              break

      return self

    def transform(self, X):
        return np.delete(X, self.dele, axis=1)


### Vif2

Vif2 realiza una selección de características basada en el VIF para eliminar características con alta multicolinealidad, lo que ayuda a mejorar la calidad de los modelos de aprendizaje automático al reducir la redundancia entre las características.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np 

class Vif2(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y = None):
        self.dele = []
        for _ in range(X.shape[1]):
            vif = np.zeros(X.shape[1])
            for i in range(X.shape[1]):
                    if i in self.dele:
                        continue
                    vif[i] = variance_inflation_factor(X, i)

            # delete the maximum one and repeat the count until the maximum value is lower than 5.
            amax  = vif.argmax()
            if vif[amax] < 5:
                break
            self.dele.append(amax)

        return self

    def transform(self, X):
        return np.delete(X, self.dele, axis=1)


Diferencia entre Vif y Vif2:

Ambas clases, Vif y Vif2, tienen el mismo propósito: realizar una selección de características basada en el factor de inflación de la varianza (VIF) para eliminar características altamente multicolineales. Sin embargo, difieren en su implementación interna:

- Iteración:

    Vif: Utiliza un bucle while para iterar y eliminar características con un VIF superior al umbral definido (5) hasta que todas las características restantes tengan un VIF inferior al umbral.

    Vif2: Utiliza un bucle for anidado para calcular el VIF de cada característica individualmente y elimina la característica con el VIF más alto en cada iteración hasta que todas las características restantes tengan un VIF inferior al umbral.

- Manipulación de datos:

    Vif: Crea una copia de los datos de entrada X y luego realiza eliminaciones en esta copia a medida que se eliminan las características con alto VIF.

    Vif2: Utiliza la matriz X original para calcular el VIF y luego elimina características de la matriz original.

- Estructura de la clase:

    Ambas clases implementan las interfaces BaseEstimator y TransformerMixin, lo que les permite ser utilizadas como estimadores y transformadores en scikit-learn.

### Correlation

Correlation realiza una selección de características basada en la significancia estadística de las diferencias entre las muestras de las clases 0 y 1 utilizando la prueba de ANOVA. Las características que no cumplen con un cierto umbral de significancia estadística se eliminan del conjunto de datos.

In [ ]:
from scipy.stats import f_oneway


class Correlation(BaseEstimator, TransformerMixin):
    def __init__(self, alpha=0.1):
        self.alpha = alpha

    def fit(self, X, y):
      X_train = X.copy()
      y_train = y.copy()

      self.dele = []
      for i in range(X_train.shape[1]):
        AnovaResults = f_oneway(X_train[y_train == 0, i], X_train[y_train == 1, i])

        if AnovaResults[1] > self.alpha:
          self.dele.append(i)


      return self

    def transform(self, X):
        return np.delete(X, self.dele, axis=1)


### Correlation2

Correlation2 se utiliza para realizar una selección de características basada en la significancia estadística de las diferencias entre las características para diferentes clases de la variable objetivo utilizando la prueba de ANOVA. Las características que no cumplen con un cierto umbral de significancia estadística se eliminan del conjunto de datos.

In [ ]:
class Correlation2(BaseEstimator, TransformerMixin):
    def __init__(self, alpha=0.1):
        self.alpha = alpha

    def fit(self, X, y):
        _, a_p      = f_oneway(X[y == 0, :], X[y == 1, :])
        self.dele   = np.flatnonzero(a_p > self.alpha)

        return self

    def transform(self, X):
        return np.delete(X, self.dele, axis=1)

Diferencias entre Correlation y Correlation2:


- Método de selección de características:

    Correlation: Utiliza el método ANOVA (Análisis de varianza) para calcular la importancia estadística de cada característica en relación con la variable objetivo. Luego, selecciona las características cuyos valores de p de ANOVA superan un umbral predefinido (alpha) para eliminarlas.

    Correlation2: Utiliza una estrategia diferente para calcular la importancia de cada característica en relación con la variable objetivo. Se basa en el método de eliminación iterativa, donde en cada iteración se calcula el VIF (Factor de inflación de la varianza) para cada característica y se elimina la característica con el VIF más alto hasta que ningún VIF sea superior a un umbral predefinido (5).

- Manipulación de datos:

    Correlation: No modifica directamente los datos de entrada X, sino que utiliza las funciones f_oneway y np.delete para calcular la importancia de las características y eliminar las características seleccionadas.

    Correlation2: Modifica los datos de entrada X durante el proceso de ajuste y transformación, eliminando directamente las características con alto VIF.

- Estructura de la clase:

    Ambas clases implementan las interfaces BaseEstimator y TransformerMixin de scikit-learn, lo que les permite ser utilizadas como estimadores y transformadores en pipelines de scikit-learn.

## Pipeline


Esta pipeline es el proceso de feature selection que se debe implementar antes de aplicar los modelos. Aqui se puede especificar el numero de feature selection, podiendo implementar uno o tantos como se quieran y en el orden que se quiera. Anteriormente se han definido funciones especificas pero aqui se especifican mas modelos de feature selection que ya estan implementados y cargados en python como IterativeImputer - para eliminar nulos - o StandardScaler - o PCA.

En este ejemplo, como solo estamos hablando de feature selection sobraria los dos primeros metodos, pero por su necesario uso en el origen se dejan para demostracion de como se deberian implementar.

In [ ]:
### PIPELINE ###

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pipeline = Pipeline([
    ('impute', IterativeImputer(random_state = 42)), #strategy to substitude the null values: mean
    ('scaler', StandardScaler()),
    ('vif', Vif2()),
    ('correlation', Correlation2()),
    ('pca', PCA()),
    ])


Un ejemplo de la implementacion del pipeline en un modelo seria, utilizando un Logistic Regression:

In [ ]:
np.random.seed(100483869) # reproducibility
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

# Logistic regression model
model = LogisticRegression()

# Put together model
clf_LR = Pipeline ([
    ( 'preprocessor', pipeline ),
    ('classifier', model)])

clf_LR_grid = clf_LR.fit(X=X_train0, y=y_train0)